In [44]:
from datetime import datetime
import json
from dateutil import parser

def converte_datetime(str_datetime: str):
    return parser.isoparse(str_datetime).replace(tzinfo=None)

def convert_data_toiso(data):
    try:
        data = datetime.strptime(data, '%d/%m/%Y %H:%M:%S')
    except ValueError:
        data = datetime.strptime(data, '%d/%m/%Y')
    return data.isoformat()





In [39]:
from sqlalchemy import Boolean, Column, DateTime, Integer, \
    String, BigInteger, ForeignKey, Numeric

class EventoBase():
    __abstract__ = True
    cnpjRecinto = Column(String(15), index=True)
    dataHoraTransmissao = Column(DateTime(), index=True)
    dataHoraCriacao = Column(DateTime(), index=True)
    codigoRecinto = Column(String(7), index=True)

    def __init__(self, cnpjRecinto, dataHoraTransmissao, dataHoraCriacao, codigoRecinto):
        self.cnpjRecinto = cnpjRecinto
        self.dataHoraTransmissao = converte_datetime(dataHoraTransmissao)
        self.dataHoraCriacao = converte_datetime(dataHoraCriacao)
        self.codigoRecinto = codigoRecinto

    def dump(self):
        dump = dict([(k, v) for k, v in vars(self).items() if not k.startswith('_')])
        # dump.pop('ID')
        dump['dataHoraTransmissao'] = datetime.strftime(self.dataHoraTransmissao,
                                                   '%Y-%m-%dT%M:%H:%S')
        dump['dataHoraCriacao'] = datetime.strftime(self.dataHoraCriacao,
                                                 '%Y-%m-%dT%M:%H:%S')
        return dump

class AcessoVeiculo():
    __tablename__ = 'acessosveiculo'
    __table_args__ = {'sqlite_autoincrement': True}
    id = Column(BigInteger().with_variant(Integer, 'sqlite'),
                primary_key=True)
    operacao = Column(String(1))
    direcao = Column(String(1))
    dataHoraOcorrencia = Column(DateTime(), index=True)
    placa = Column(String(7))
    codigoRecinto = Column(String(7), index=True)
    def __init__(self, operacao, direcao, dataHoraOcorrencia, placa, codigoRecinto):
        self.operacao = operacao
        self.direcao = direcao
        self.dataHoraOcorrencia = converte_datetime(dataHoraOcorrencia)
        self.placa = placa
        self.codigoRecinto = codigoRecinto
    
    def get_campos():
        return [k for k, v in vars(AcessoVeiculo).items() if '_' not in k]
    
    def dump(self):
        dump = dict([(k, v) for k, v in vars(self).items() if not k.startswith('_')])
        # dump.pop('ID')
        dump['dataHoraOcorrencia'] = datetime.strftime(self.dataHoraOcorrencia,
                                                   '%Y-%m-%dT%M:%H:%S')
        return dump


    


# Carrega JSON

In [7]:
with open('data\consultaGenerica\json.txt') as json_in:
    texto = json_in.readlines()
json_raw = json.loads(''.join(texto))
json_raw

[{'dadosTransmissao': {'versaoJsonEvento': '1.2.5',
   'cnpjRecinto': '45050663000906',
   'dataHoraTransmissao': '2023-06-14T15:45:55.000-0300',
   'dataHoraCriacao': '2023-06-14T15:45:55.278286-0300',
   'protocoloRecepcao': 'ece7d7fe-4a77-4043-be77-c4925289cf97',
   'idCriacao': 147370125,
   'traducaoTipoEvento': 'Controle de Agendamento/Acesso de VeÃ\xadculos',
   'traducaoPerfilAtuacao': 'DepositÃ¡rio',
   'tipoEvento': 1,
   'codigoRecinto': '8931342',
   'cpfRepresentanteLegal': '88071367834',
   'hash': 'a77bba48c7af0a7749d6313a61fd951',
   'ipRemetente': '177.25.34.194',
   'perfilAtuacao': 7},
  'protocolosCadeia': {'folha': 'ece7d7fe-4a77-4043-be77-c4925289cf97'},
  'situacaoEvento': 'Evento recepcionado',
  'jsonOriginal': {'direcao': 'S',
   'ocrPlaca': False,
   'listaChassi': [],
   'dataHoraOcorrencia': '2023-06-14T15:40:00.000-0300',
   'listaNfe': [],
   'listaDeclaracaoAduaneira': [{'tipo': 'DTA',
     'traducaoTipo': 'DeclaraÃ§Ã£o de TrÃ¢nsito Aduaneiro',
     'idE

In [19]:
evento_0 = json_raw[0]['jsonOriginal']

In [20]:
evento_0

{'direcao': 'S',
 'ocrPlaca': False,
 'listaChassi': [],
 'dataHoraOcorrencia': '2023-06-14T15:40:00.000-0300',
 'listaNfe': [],
 'listaDeclaracaoAduaneira': [{'tipo': 'DTA',
   'traducaoTipo': 'DeclaraÃ§Ã£o de TrÃ¢nsito Aduaneiro',
   'idElemento': 'DocDesembaraco_1393677',
   'numeroDeclaracao': '2302373760'}],
 'traducaoModal': 'RodoviÃ¡rio',
 'codigoRecinto': '8931342',
 'modal': 'R',
 'placa': 'CUA5137',
 'listaConteineresUld': [{'listaNavio': [{'imo': '09706190',
     'nome': 'CAPE AKRITAS',
     'idElemento': 'Navio_47691'}],
   'listaPortoDescarregamento': [{'porto': 'BRSSZ',
     'idElemento': 'Porto_3204',
     'traducaoPorto': 'SANTOS'}],
   'tipo': '22G1',
   'listaCliente': [{'cnpjCliente': '07810597000189',
     'idElemento': 'Empresa_354185',
     'nomeCliente': 'ArmazÃƒÂ©m do Nono ImportaÃƒÂ§ÃƒÂ£o e ComÃƒÂ©rcio de Especialidades Ltda. EPP'}],
   'traducaoTipo': 'PASSIVE UPPER VENTS(20/86)',
   'listaLacres': [{'tipo': 'EST',
     'traducaoTipo': 'Estrangeiro',
     'num

In [36]:
evento_filtrado = {k: v for k, v in evento_0.items() if k in AcessoVeiculo.get_campos()}

In [37]:
evento_filtrado

{'direcao': 'S',
 'dataHoraOcorrencia': '2023-06-14T15:40:00.000-0300',
 'codigoRecinto': '8931342',
 'placa': 'CUA5137',
 'operacao': 'C'}

In [45]:
acessoveiculo = AcessoVeiculo(**evento_filtrado)

In [46]:
acessoveiculo

In [47]:
acessoveiculo.dump()

{'operacao': 'C',
 'direcao': 'S',
 'dataHoraOcorrencia': '2023-06-14T40:15:00',
 'placa': 'CUA5137',
 'codigoRecinto': '8931342'}

In [102]:
eventos_api = pd.DataFrame(eventos)
eventos_api.head()

,dataHoraTransmissao,dataHoraOcorrencia,direcao,placa,container
0,2023-06-14 15:45:55,2023-06-14 15:40:00,S,CUA5137,CAIU2651169
1,2023-06-14 15:57:49,2023-06-14 15:52:00,S,FYX4F95,KOCU5089390
2,2023-06-14 16:03:50,2023-06-14 15:58:00,S,EXP1234,None
3,2023-06-14 16:03:51,2023-06-14 15:58:00,E,CKK8557,None
4,2023-06-14 16:03:52,2023-06-14 15:59:00,E,FAT6I21,None
